In [259]:

import pandas as pd
import numpy as np
# example of one hot encoding for a neural network
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from keras.models import Sequential
from keras.layers import Dense
# Scaling
from sklearn.preprocessing import RobustScaler

# Train Test Split
from sklearn.model_selection import train_test_split

# Models
import torch
import torch.nn as nn
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Metrics
from sklearn.metrics import accuracy_score, classification_report, roc_curve

# Cross Validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

print('Packages imported...')

Packages imported...


In [260]:
# load the train dataset
def load_dataset(train, test, evaluation):
	# load the dataset as a pandas DataFrame
	print("load the dataset as a pandas DataFrame")
	train = pd.read_csv(train)
	test = pd.read_csv(test)
	evaluation = pd.read_csv(evaluation)

	train = train.replace(np.nan, 'none').astype(str)
	test = test.replace(np.nan, 'none').astype(str)
	evaluation = evaluation.replace(np.nan, 'none').astype(str)

	print(train.info())

    # select columns
	print("select columns")
	train_df = train[['consumed_part_name','pfmea_potential_effect_of_failure','pfmea_severity']]
	print(train_df)
	# train_df['consumed_part_name'] = train_df['consumed_part_name'].fillna('0')
	# train_df['pfmea_potential_effect_of_failure'] = train_df['pfmea_potential_effect_of_failure'].fillna('0')
	# train_df['pfmea_severity'] = train_df['pfmea_severity'].fillna("0")
	test_df = test[['consumed_part_name','pfmea_potential_effect_of_failure','pfmea_severity']]
	print(test_df)
	# test_df['consumed_part_name'] = test_df['consumed_part_name'].fillna('0')
	# test_df['pfmea_potential_effect_of_failure'] = test_df['pfmea_potential_effect_of_failure'].fillna('0')
	# test_df['pfmea_severity'] = test_df['pfmea_severity'].fillna("0")
	evaluation_df = evaluation[['consumed_part_name','pfmea_potential_effect_of_failure','pfmea_severity']]
	print(evaluation_df)
	# evaluation_df['consumed_part_name'] = evaluation_df['consumed_part_name'].fillna('0')
	# evaluation_df['pfmea_potential_effect_of_failure'] = evaluation_df['pfmea_potential_effect_of_failure'].fillna('0')
	# evaluation_df['pfmea_severity'] = evaluation_df['pfmea_severity'].fillna("0")


	# retrieve numpy array
	print("retrieve numpy array")
	#Considering y variable holds numpy array
	# dataset_train = tf.convert_to_tensor(train_df, dtype=tf.int64)
	# dataset_test = tf.convert_to_tensor(test_df, dtype=tf.int64)
	# evaluation_test = tf.convert_to_tensor(evaluation_df, dtype=tf.int64)
	dataset_train = train_df
	dataset_test = test_df
	evaluation_test = evaluation_df
	# dataset_train = train_df.values
	# dataset_test = test_df.values
	# evaluation_test = evaluation_df.values

	# Define Features and Label
	feature = ['consumed_part_name','pfmea_potential_effect_of_failure']
	label = ['pfmea_severity']

	# split into input (X) and output (y) variables
	print("split into input (X) and output (y) variables")
	X_train = dataset_train[feature].values
	y_train = dataset_train[label].values

	X_test = dataset_test[feature].values
	y_test = dataset_test[label].values

	X_evaluation = evaluation_test[feature].values
	y_evaluation = evaluation_test[label].values


	# # format all fields as string
	# print("format all fields as string")
	# X_train = X_train.astype(str)
	# X_test = X_test.astype(str)
	# X_evaluation = X_evaluation.astype(str)
	# y_train = y_train.astype(str)
	# y_test = y_test.astype(str)
	# y_evaluation = y_evaluation.astype(str)

	# reshape target to be a 2d array
	print("reshape target to be a 2d array")
	y_train = y_train.reshape((len(y_train), 1))
	y_test = y_test.reshape((len(y_test), 1))
	y_evaluation = y_evaluation.reshape((len(y_evaluation), 1))

	print("Done")


	return X_train, y_train, X_test, y_test, X_evaluation, y_evaluation

In [261]:
# prepare input train data
def prepare_inputs(X_train, X_test, X_evaluation):
	print("prepare input train data")
	ohe = OrdinalEncoder()
	ohe.fit(X_train)
	X_train_enc = ohe.fit_transform(X_train)
	X_test_enc = ohe.fit_transform(X_test)
	X_evaluation_enc = ohe.fit_transform(X_evaluation)
	print("Done")
	return X_train_enc, X_test_enc, X_evaluation_enc

In [262]:
# prepare target
def prepare_targets(y_train, y_test, y_evaluation):
	print("prepare target")
	le = LabelEncoder()
	le.fit(y_train)
	y_train_enc = le.transform(y_train)
	y_test_enc = le.transform(y_test)
	y_evaluation_enc = le.transform(y_evaluation)
	print("Done")
	return y_train_enc, y_test_enc, y_evaluation_enc

In [263]:
train = 'training.csv'
test = 'test.csv'
evaluation = 'evaluation.csv'

In [264]:
# load the dataset
X_train, y_train, X_test, y_test, X_evaluation, y_evaluation = load_dataset(train, test, evaluation)

# prepare input data
X_train_enc, X_test_enc,X_evaluation_enc = prepare_inputs(X_train, X_test, X_evaluation)

# prepare output data
y_train_enc, y_test_enc, y_evaluation_enc = prepare_targets(y_train, y_test, y_evaluation)



load the dataset as a pandas DataFrame


/var/folders/9z/xcg1b2s55v36mhfj9jw2fwsh0000gn/T/ipykernel_29841/1475370861.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(train)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196782 entries, 0 to 196781
Data columns (total 30 columns):
 #   Column                               Non-Null Count   Dtype 
---  ------                               --------------   ----- 
 0   plant                                196782 non-null  object
 1   line_mrn                             196782 non-null  object
 2   sub_process_station                  196782 non-null  object
 3   task_number                          196782 non-null  object
 4   consumed_part_number                 196782 non-null  object
 5   task_list_description                196782 non-null  object
 6   consumed_part_top_number             196782 non-null  object
 7   consumed_part_name                   196782 non-null  object
 8   consumed_part_state                  196782 non-null  object
 9   consumed_part_description            196782 non-null  object
 10  consumed_part_assembly_instructions  196782 non-null  object
 11  consumed_part_top_name    

/Users/ananya_pramanik/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ananya_pramanik/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: y contains previously unseen labels: 'none'

In [255]:
print(X_train.shape, X_test.shape, X_evaluation.shape)
print(y_train.shape, y_test.shape, y_evaluation.shape)


(196782, 2) (31845, 2) (63689, 2)
(196782, 1) (31845, 1) (63689, 1)


In [256]:
# prepare input data
X_train_enc, X_test_enc,X_evaluation_enc = prepare_inputs(X_train, X_test, X_evaluation)
print(X_train_enc.shape, X_test_enc.shape, X_evaluation_enc.shape)
print(X_train_enc)


prepare input train data
Done
(196782, 2) (31845, 2) (63689, 2)
[[633.   7.]
 [633.   7.]
 [167.   2.]
 ...
 [780.  21.]
 [780.  18.]
 [550.  21.]]


In [257]:
# prepare output data
y_train_enc, y_test_enc, y_evaluation_enc = prepare_targets(y_train, y_test, y_evaluation)
print(y_train_enc.shape, y_test_enc.shape, y_evaluation_enc.shape)

prepare target


/Users/ananya_pramanik/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ananya_pramanik/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: y contains previously unseen labels: 'none'

In [248]:
print('X_train',X_train.shape)
print('y_train',y_train.shape)
print('X_test',X_test.shape)
print('y_test',y_test.shape)
print('X_evaluation',X_evaluation.shape)
print('y_evaluation',y_evaluation.shape)

print('X_train_enc',X_train_enc.shape)
print('X_test_enc',X_test_enc.shape)
print('X_evaluation_enc',X_evaluation_enc.shape)

print('y_train_enc',y_train_enc.shape)
print('y_test_enc',y_test_enc.shape)
print('y_evaluation_enc',y_evaluation_enc.shape)

X_train (196782, 2)
y_train (196782, 1)
X_test (31845, 2)
y_test (31845, 1)
X_evaluation (63689, 2)
y_evaluation (63689, 1)
X_train_enc (196782, 2)
X_test_enc (31845, 2)
X_evaluation_enc (63689, 2)
y_train_enc (222911,)
y_test_enc (31845,)
y_evaluation_enc (63689,)


In [249]:
# instantiate the classifier
gbt = GradientBoostingClassifier(n_estimators = 1, max_depth=1,subsample=0.8,max_features=0.2,random_state=42)

# fitting the model
gbt.fit(X_train_enc, y_train_enc)

# predicting values
y_pred = gbt.predict(X_evaluation_enc)
print("The test accuracy score of Gradient Boosting Classifier is ", accuracy_score(y_test_enc, y_pred))

ValueError: Found input variables with inconsistent numbers of samples: [196782, 222911]